# Tensorflow Basic Concept

## 关于Tensorflow

TF核心是**数据流图（Data flow graphs）**，可用于**数值计算**。其图中的节点（Node）表示数学操作；边（Edge）则表示多维数据的流动，即张量（Tensor）。

TF的特性：

- **可移植性（Portability）**：在CPU或GPU上运行，支持台式机，服务器，手机等设备
- **性能最优化**：将硬件的计算潜能全部发挥出来
- **自动求微分**：对于基于梯度的学习算法可自动求微分

## 一般使用步骤

1. **构建图**
2. **计算误差**
3. **选择一个优化器来训练网络**
4. **执行图计算**

下面展示了一个简单的例子：

In [3]:
import tensorflow as tf
import numpy as np

# 生成假数据, 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构建图（线性模型）
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 计算误差（最小化方差）
loss = tf.reduce_mean(tf.square(y - y_data))

# 选择优化器训练网络
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 执行图计算
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(W), sess.run(b))

0 [[0.334971   0.25883147]] [0.28183356]
20 [[0.14209764 0.20081656]] [0.27774435]
40 [[0.1093827  0.19963202]] [0.29531085]
60 [[0.10209645 0.19980331]] [0.2990087]
80 [[0.10046954 0.19993219]] [0.2997897]
100 [[0.10010538 0.19997986]] [0.29995522]
120 [[0.1000237  0.19999444]] [0.29999042]
140 [[0.10000534 0.19999854]] [0.29999793]
160 [[0.10000122 0.19999963]] [0.29999956]
180 [[0.10000027 0.19999991]] [0.29999992]
200 [[0.1000001  0.19999997]] [0.29999995]


### 构建图

TF使用图 (graph) 来表示计算任务，图中的节点被称之为**op（operation）**，一个 op获得0个或多个`Tensor`, 执行计算, 产生0个或多个`Tensor`。每个`Tensor`是一个类型化的多维数组，例如, 你可以将一小组图像集表示为一个四维浮点数数组, 这四个维度分别是`[batch, height, width, channels]`。

Python库有一个**默认图 (default graph)**，如果没有显式的声明新图，那么构建的op会被放入默认图。

如下例子里面包含3个op：两个`constant()op`, 和一个`matmul()op`，其中`constant()op`也被称为`源op（source op）`。

In [5]:
# 构造器的返回值代表该常量 op 的返回值.
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量 op, 产生一个 2x1 矩阵.
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法 matmul op , 把 'matrix1' 和 'matrix2' 作为输入.
# 返回值 'product' 代表矩阵乘法的结果.
product = tf.matmul(matrix1, matrix2)

### 执行图计算

**构造阶段完成后, 才能启动图**. 启动图的第一步是创建一个`Session`对象, 如果无任何创建参数, 会话构造器将启动默认图. 其中`sess.run(op_output)`表示我想取某个op的输出值。

如下例子执行了图：函数调用`run(product)`会触发了图中三个op(两个常量op和一个矩阵乘法op)的执行。

In [12]:
# 'product'代表了矩阵乘法op的输出, 传入它是向方法表明, 我们希望取回矩阵乘法op的输出.
with tf.Session() as sess:
  res = sess.run(product)
  print(res)

[[12.]]


在实现上, TensorFlow将图形定义转换成**分布式执行的操作**，以充分利用可用的计算资源（如 CPU 或 GPU），一般你不需要显式指定使用 CPU 还是 GPU。你可以使用类似`with tf.device("/gpu:1"):`的方式显式的定义某个设备来执行。

### 交互式使用

使用`sess = tf.InteractiveSession()`可以建立类似IPython之类的Python交互环境。使用`Tensor.eval()`和`Operation.run()`方法代替`Session.run()`, 这样可以避免使用一个变量来持有会话。

In [10]:
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])
add_ = tf.add(x, a)

init=tf.global_variables_initializer()
init.run()

print(x.eval())
print(add_.eval())

[1. 2.]
[4. 5.]


/Users/niudong/workon_home/py3/lib/python3.6/site-packages/tensorflow/python/client/session.py:1714: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
